## Uczenie modeli

Niniejszy skrypt służy w celu imitowania konsoli i dyskusji na temat skryptu do eksperymentów i uczenia modeli - `train.py`. Opisane w niej również będą użyte techniki MLOps oraz zastosowane praktyki CI/CD. 

Proces stworzenia dobrego skryptów/systemu z CI/CD (w zależności od używanych technologii) jest czasochłonny i wymaga przemyślenia wielu koncepcji. Na przykłądzie zadania 6.2 z PolEval chciałbym pokazać tylko pewną część działania, na podstawie której chciałbym podyskutować i omówić co chciałbym zrobić w dalszych krokach.

Z racji tego, że pracuje lokalnie, tzn. na moim prywatnym laptopie, decyduje się na użycie [MLflow](https://mlflow.org/).

Tak jak mówiłem, niniejszy Notebook służy jako imitację konsoli, a pod spodem widzimy log z operacji wykonywanych w zdefiniowanym przeze mnie ML Pipeline.

In [3]:
!python train.py

2021-07-20 09:31:30,759 - INFO - Collecting new data
2021-07-20 09:31:30,782 - INFO - New data collected
2021-07-20 09:31:30,783 - INFO - Data verification and validation
2021-07-20 09:31:30,835 - INFO - Starting MLFlow run: Support Vector Classifier v01
2021-07-20 09:31:30,857 - INFO - Creating validation set
2021-07-20 09:31:30,868 - INFO - Creating cleaning pipeline
2021-07-20 09:31:30,891 - INFO - Fitting cleaning pipeline and transforming training set
2021-07-20 09:31:32,388 - INFO - Transforming validation set
2021-07-20 09:31:33,031 - INFO - Performing hyperparatemer optimization
2021-07-20 09:31:33,035 - INFO - Hyperparatemer optimization completed. Best params are: tfidf: {'encoding': 'utf-8', 'decode_error': 'strict', 'strip_accents': None, 'lowercase': False, 'preprocessor': None, 'tokenizer': None, 'analyzer': 'word', 'stop_words': None, 'token_pattern': '(?u)\\b\\w\\w\\w+\\b|[^\\x00-\\x7F]', 'ngram_range': (1, 1), 'max_df': 0.95, 'min_df': 5} ml model: {'C': 1.0, 'kernel':

Jest on przykładem działania pipeline'a, w którym nie ma jeszcze zaimplementowanych wszystkich elementów aby móc powiedzieć o pełnym CI/CD, o których teraz chciałbym wspomnieć:

1. Data versioning - wersjonowanie danych, by można było określić na jakich zbiorach modele były uczone i testowane. MLFlow pozwala mi na zrzut danych jako artefaktów i w każdym przebiegu pipeline są one zapisywane. 
2. Data verification/validation - innymi słowy, badanie jakości danych na których uczymy model i porównanie ich z danymi uczacymi na ktorych uczony był model na produkcji. W tym etapie możnaby zbadać covariate shift, tzn. różnice w rozkładach słów i emotikonek między danymi użytymi do nauki tego modelu vs. dane użyte do modeli na produkcji oraz concept drift, tzn. jak rozkłady kategorii *non-harmful*, *hate-speech* i *cyberbullying* uległy zmianie w porównianu do zbiorów uczących modelu na produkcji.
3. Model analysis/evaluation - Czyli nauka modelu wraz z optymalizacją hiperparametrów. Tego etapu jeszcze brak, użyłbym w nim [optuny i jej intergracji z MLflow](https://optuna.readthedocs.io/en/stable/reference/generated/optuna.integration.MLflowCallback.html)
4. Model validation - Porównanie z obecnym modelem na produkcji oraz z baselinem. Obecnie zaimplementowane jest tylko porównanie na bazie miary F1 micro-average score. To co dodałbym jeszcze to porównanie czasu inferencji oraz "zasobożerność", tzn. ile pamięci potrzeba, zużycie CPU (lub GPU w zależności od testowanego modelu).

MLFlow pozwala na zarządzanie metadanymi i rejestrem - ze wszystkich etapów zrzucane są użyte dane, pipeline'y do czyszczenia i uczenia modelu, wartości miar na bazie których porównywano model wraz z parametrami (w folderze `./mlruns/`). W wyniku przebiegu otrzymujemy wszystkie elementy niezbędne do wdrożenia zbudowanego za pomocą skryptu `train.py` modelu.

Scenariusze kiedy mógłby być używany pipeline:

1. Ręcznie, podczas eksperymentowania i pracy nad kolejnymi rozwiązaniami
2. Zgodnie z przyjętym harmonogramem - uczymy model co pewien określony czas
3. Gdy "pojawią się" nowe dane
4. Gdy jakość modelu drastycznie spadła (np. o określoną wartość f1 micro-average)
5. Gdy obserwowane są duże zmiany w rozkładach słów (covariate shift) i/lub rozkładach kategorii (concept drift).

Do punktu 3., 4. i 5. niezbędne byłoby stworzenie cyklicznego monitoringu jakości modelu i danych.

W kolejnym notebooku - `deployment.ipynb` (który podobnie jak ten notebook) imituje konsole, omówimy proces wdrożenia rozwiązania, które stworzyliśmy korzystając z `train.py`.